In [1]:
import gym, gym_2048
from gym_2048.envs.game_2048 import Game2048

import math, random
import numpy as np
from copy import deepcopy
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

## Memory Replay

In [2]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

## DQN

In [129]:
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,padding=1), nn.ReLU())
        self.conv2 = nn.Sequential(nn.Conv2d(in_channels=16,out_channels=64,kernel_size=2), nn.ReLU())
        self.conv3 = nn.Sequential(nn.Conv2d(in_channels=64,out_channels=128,kernel_size=2), nn.ReLU())
        self.dense = nn.Sequential(nn.Linear(512,64), nn.ReLU())
        self.action = nn.Linear(64,4)
        
    def forward(self,x):
        output_conv = self.conv3(self.conv2(self.conv1(x)))
        output_flat = output_conv.view(output_conv.shape[0],-1)
        output_action = self.action(self.dense(output_flat))
        return output_action

## Training RL agent

In [165]:
BATCH_SIZE = 256
GAMMA = 0.2
EPS_START = 0.95
EPS_END = 0.5
EPS_DECAY = 1000
TARGET_UPDATE = 10

In [176]:
policy_net = DQN()
#policy_net.load_state_dict(torch.load("target_net_new_record_9000.pt"))
target_net = DQN()
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.RMSprop(policy_net.parameters(),lr=0.00003)
memory = ReplayMemory(50000)

steps_done = 0
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row. second column on max result is index of where max element was found
            q = policy_net(torch.from_numpy(state)[None,None,])
            action_sorted = np.argsort(q.numpy())
            action_sorted = torch.from_numpy(action_sorted)[0]
            #print(action_sorted)
            for i in range(4):
                if env.moves_available()[action_sorted[i]] == False:
                    continue
                else:
                    return torch.Tensor([action_sorted[i]])[None,]
    else:
        # if random.random() < -1:
        #     return torch.tensor([[random.randrange(4)]], dtype=torch.long)
        # else:
        action_sorted = np.argsort(torch.Tensor([4,3,2,1]).numpy())
        action_sorted = torch.from_numpy(action_sorted)
        for i in range(4):
            if env.moves_available()[action_sorted[i]] == False:
                continue
            else:
                return torch.Tensor([action_sorted[i]])[None,]

def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), dtype=torch.bool)
    non_final_next_states = torch.cat([torch.tensor(s) for s in batch.next_state if s is not None])
    state_batch = torch.cat((torch.Tensor(batch.state),))
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_batch = state_batch[None,].reshape(BATCH_SIZE,1,4,4)
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    #print(state_batch)
    state_action_values = policy_net(state_batch).gather(1, action_batch.type(torch.int64))

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE)
    non_final_next_states = non_final_next_states[None,].reshape(-1,1,4,4)
    #print(non_final_next_states)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    if steps_done % 100 == 0:
        print(loss)
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [177]:
env = Game2048()
episode_durations = []
num_episodes = 10000
highest = 0
for i_episode in range(num_episodes):
    if i_episode % 10 == 0:
        print(i_episode)
    # Initialize the environment and state
    env._reset()
    state = env._get_state()
    #print(state)
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, info = env._step(action.item())
        reward = torch.tensor([reward])

        # Observe new state
        if not done:
            next_state = env._get_state()
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            if info['score'] > highest:
                highest = info['score']
                print("new record:", i_episode,info['score'])
                print(env._get_state())
            #plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
    
    if (i_episode+1) % 100 == 0:
        print(f"Episode: {i_episode}, {info['score']}")
        print(env._get_state())
print('Complete')
env._render()
env.close()

0
new record: 0 2064
[[  2.   8.   4.   2.]
 [  4.  32.   8.   4.]
 [ 16. 128.  32.   8.]
 [ 64.   8.   2.  16.]]
tensor(7.2090, grad_fn=<SmoothL1LossBackward>)
tensor(6.4516, grad_fn=<SmoothL1LossBackward>)
tensor(6.1664, grad_fn=<SmoothL1LossBackward>)
tensor(6.2475, grad_fn=<SmoothL1LossBackward>)
tensor(6.0288, grad_fn=<SmoothL1LossBackward>)
new record: 6 2768
[[  4.   8.  16.   8.]
 [ 64.   4.   2.   4.]
 [  4.  64. 128.   2.]
 [ 32.   2.   8.  32.]]
tensor(6.2438, grad_fn=<SmoothL1LossBackward>)
tensor(6.2907, grad_fn=<SmoothL1LossBackward>)
tensor(6.4857, grad_fn=<SmoothL1LossBackward>)
10
tensor(6.8028, grad_fn=<SmoothL1LossBackward>)
tensor(6.5698, grad_fn=<SmoothL1LossBackward>)
tensor(6.4248, grad_fn=<SmoothL1LossBackward>)
tensor(6.4542, grad_fn=<SmoothL1LossBackward>)
tensor(8.1579, grad_fn=<SmoothL1LossBackward>)
tensor(5.6503, grad_fn=<SmoothL1LossBackward>)
tensor(5.4505, grad_fn=<SmoothL1LossBackward>)
tensor(5.8251, grad_fn=<SmoothL1LossBackward>)


KeyboardInterrupt: 

In [ ]:
target_net = DQN()
target_net.load_state_dict(torch.load("target_net_new_record_9000.pt"))

In [139]:
action_sorted = np.argsort(torch.Tensor([4,3,1,2]).numpy())
action_sorted
# 0 up, 1 right, 2 down, 3 left
# down left right up
# 2 3 1 0

array([2, 3, 1, 0])

## Evaluating RL agent

In [148]:
target_net.eval()
best = 0
best_env = 0
sum = 0
f = open('out11.txt', 'w')

for i in range(1):
    env = Game2048()
    env._reset()
    env._render()
    print("\n")

    done = False
    moves = 0
    total = 0
    merge_count = 0
    while not done:
        q = target_net(torch.from_numpy(env._get_state()[None,None,]).float())
        action_sorted = np.argsort(q.detach().numpy())[0]
        action_sorted = np.argsort([4,3,1,2])
        #print(action_sorted)
        #action = [0,1,2,3][torch.argmax(q)]
        for i in range(4):
            if env.moves_available()[action_sorted[i]] == False:
                continue
            else:
                action = action_sorted[i]
                break
        #if (env.moves_available()[action] == False):
        #    action = np.random.choice([i for indx,i in enumerate([0,1,2,3]) if env.moves_available()[indx]])
        merge_count = env.merge_count()
        next_state, reward, done, info = env._step(action)
        #print(reward)
        moves += 1
        total += reward
        # if total > 0:    
            # print(q, file=f)
            # print(moves,info,done,np.sum(merge_count), file=f)
            # print(env._get_state(), file=f)
        print(moves,info,done,np.sum(merge_count))
        print(env._get_state())
        #print("\n")
        #env._render()
        #print("\n")
        if total > best:
            best = total
            best_env = deepcopy(env)
    #print('Total Moves: {}'.format(moves))
    sum += total
    env.close()
#print(best_env._get_state())
#print(best, moves)
#print(sum/(i+1))
#print("Done")


1 {'score': 0, 'action': 'down', 'won': False} False 0
[[0. 0. 0. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 2.]]
2 {'score': 4, 'action': 'down', 'won': False} False 1
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]
 [2. 0. 0. 4.]]
3 {'score': 8, 'action': 'down', 'won': False} False 1
[[0. 0. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 0.]
 [4. 0. 0. 4.]]
4 {'score': 8, 'action': 'down', 'won': False} False 0
[[2. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [4. 0. 2. 4.]]
5 {'score': 8, 'action': 'down', 'won': False} False 0
[[0. 0. 0. 0.]
 [0. 2. 0. 0.]
 [2. 0. 0. 0.]
 [4. 0. 2. 4.]]
6 {'score': 8, 'action': 'down', 'won': False} False 0
[[0. 0. 0. 0.]
 [0. 0. 0. 2.]
 [2. 0. 0. 0.]
 [4. 2. 2. 4.]]
7 {'score': 8, 'action': 'down', 'won': False} False 0
[[4. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 2.]
 [4. 2. 2. 4.]]
8 {'score': 8, 'action': 'down', 'won': False} False 0
[[0. 0. 0. 0.]
 [4. 0. 0. 2.]
 [2. 0. 0. 2.]
 [4. 2. 2. 4.]]
9 {'score': 12, 'action': 'down', 'won': False} False 1
[[2. 0. 0. 0.]


In [70]:
torch.save(target_net.state_dict(), "target_net_new_record_9000.pt")

In [73]:
np.argsort(torch.Tensor([3,4,2,1]).numpy())

array([3, 2, 0, 1])